In [9]:
import numpy as np
import cv2
#import pandas as pd
import os
import tifffile as tiff
from PIL import Image
import skimage
import czifile
import matplotlib.pyplot as plt
from aicsimageio import AICSImage

## CZI to TIFF

In [37]:
def czi_to_tiff(czi_folder,save_path):
    """read czi files and save as tiff files"""
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for root, dirs, files in os.walk(czi_folder):
        for file in files:
            if file.endswith('.czi'):
                czi = AICSImage(os.path.join(root,file))
                image = czi.get_image_data("CZYX", S=0, T=0, Z=0)
                image = np.squeeze(image)
                image = np.moveaxis(image, 0, -1)
                tiff.imwrite(os.path.join(save_path,file.replace('.czi','.tif')),image)

czi_to_tiff(czi_folder='/Users/tom/Downloads/BZ_OL_images',
            save_path='/Users/tom/Downloads/BZ_OL_images_tiff2')


BZ-MBL_002_20x_LatA500nM-right-19.czi
BZ-MBL_002_20x_DMSO-left-02_photogenic.czi
BZ-MBL_002_20x_LatA2uM-left-23.czi
BZ-MBL_002_20x_LatA500nM-right-18.czi
BZ-MBL_002_20x_ionomycin-left-40.czi
BZ-MBL_002_20x_LatA2uM-right-29.czi
BZ-MBL_002_20x_LatA2uM-left-24.czi
BZ-MBL_002_20x_LatA2uM-right-28.czi
BZ-MBL_002_20x_LatA2uM-left-25.czi
BZ-MBL_002_20x_LatA500nM-right-22.czi
BZ-MBL_002_20x_ionomycin-left-37_photogenic.czi
BZ-MBL_002_20x_LatA500nM-right-20.czi
BZ-MBL_002_20x_LatA2uM-left-27.czi
BZ-MBL_002_20x_LatA2uM-left-26.czi
BZ-MBL_002_20x_LatA500nM-right-21.czi
BZ-MBL_002_20x_ionomycin-left-34.czi
BZ-MBL_002_20x_DMSO-left-03.czi
BZ-MBL_002_20x_ionomycin-left-35.czi
BZ-MBL_002_20x_ionomycin-left-36.czi
BZ-MBL_002_20x_DMSO-left-04.czi
BZ-MBL_002_20x_DMSO-right-10.czi
BZ-MBL_002_20x_DMSO-right-11.czi
BZ-MBL_002_20x_DMSO-left-05.czi
BZ-MBL_002_20x_ionomycin-left-33_photogenic.czi
BZ-MBL_002_20x_DMSO-right-07.czi
BZ-MBL_002_20x_DMSO-left-06.czi
BZ-MBL_002_20x_ionomycin-right-46.czi
BZ-MBL_002_

## OMETIFF to TIFF

In [61]:
input_path ='/Users/tom/Desktop/Stanford/RA/OligodendroSight/datasets/Lauren/Sample_TomGUI/1.czi'
img = AICSImage(input_path)
img_data = img.get_image_data("CZYX", S=0, T=0, Z=0)
print(img_data.shape)

(4, 1, 2208, 2752)


In [13]:
def ometifs_to_tifs(dir_path, save_path):
    """converts ome-tifs to tifs"""
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for file in os.listdir(dir_path):
        if file.endswith('.ome.tif'):
            img = skimage.io.imread(os.path.join(dir_path, file))
            for i in range(img.shape[0]):
                skimage.io.imsave(os.path.join(save_path, file[:-8] + f'_{i}.tif'), img[i])

input_path = '/Users/tom/Downloads/ometifs'
output_path = '/Users/tom/Downloads/tifs_from_ome'

ometifs_to_tifs(input_path, output_path)

## Normalize and convert to grayscale

In [68]:

folder_path = "/Users/tom/Desktop/Stanford/RA/OligodendroSight/datasets/Lauren/Sample_TomGUI_tiff"
save_path_norm = "/Users/tom/Desktop/Stanford/RA/OligodendroSight/datasets/Lauren/Sample_TomGUI_norm"
GRAYSCALE = True
if not os.path.exists(save_path_norm):
    os.makedirs(save_path_norm)
for file in os.listdir(folder_path):
    if file.endswith('.tif'):
        original_img = skimage.io.imread(os.path.join(folder_path, file))
        if GRAYSCALE:
            float_gray_img = skimage.io.imread(os.path.join(folder_path, file), as_gray=True)
            float_gray_img_norm = skimage.exposure.equalize_adapthist(float_gray_img, clip_limit=0.04)
            img_norm = np.stack(((float_gray_img_norm * 255).astype(np.uint8),)*3, axis=-1)
        else:
            norm_channels = []
            for i in range(original_img.shape[2]):
                if np.max(original_img[:,:,i]) > 0:
                    norm_channel = skimage.exposure.equalize_adapthist(original_img[:,:,i], clip_limit=0.03)
                    norm_channel = (norm_channel * 255).astype(np.uint8)
                else:
                    norm_channel = original_img[:,:,i]
                norm_channels.append(norm_channel)
            img_norm = np.stack(norm_channels, axis=-1)
        skimage.io.imsave(os.path.join(save_path_norm, os.path.basename(file)), img_norm)
print(f'{len(os.listdir(save_path_norm))} images were normalized and saved as grayscale.')

22 images were normalized and saved as grayscale.
